In [51]:
import os
import re
import random
import time
import numpy as np
import torch
import math
import glob
from time import perf_counter

from resnet_oc.resnet_oc import get_resnet34_oc
from resnet_oc_mod.resnet_oc_mod import get_resnet34_oc_mod
from resnet_oc.resnet_oc import get_resnet34_oc

from mapillary import mapillary
from torch.utils.data import DataLoader
from argparse import ArgumentParser
from transform import Colorize

from iouEval import iouEval
import wandb
import torch.onnx

In [52]:
def load_checkpoint(model_path):
    #Must load weights, optimizer, epoch and best value.
    file_resume = f'{model_path}'
    #file_resume = savedir + '/model-{}.pth'.format(get_last_state(savedir))
    assert os.path.exists(file_resume), "No model checkpoint found"
    checkpoint = torch.load(file_resume)

    return checkpoint

In [60]:
model = get_resnet34_oc(pretrained_backbone=True)
model = model.cuda()

#checkpoint = load_checkpoint('save/resnet_oc_lw/model-12.pth')
#model.load_state_dict(checkpoint['model'])

Using pretrained backbone: True


In [64]:
for item in model.backbone.modules():
    print(item)

rack_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (layer3): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (downsample): Sequential(
        (0): Conv2d(128, 256, kernel_size=(1, 1), stride=(2, 2), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): BasicBlock(
      (conv1): Conv2d(256, 256, kernel_size=(3, 3

In [54]:
dataset_val = mapillary('/media/alexey/HDD/Mapillary', 'val', height=1080, part=0.1) # Taking only 10% of images
loader_val = DataLoader(dataset_val, num_workers=4, batch_size=1, shuffle=False)
print('Loaded', len(loader_val), 'files')

/media/alexey/HDD/Mapillary/val/1920_1080/images
Loaded 200 files


In [55]:
x = next(iter(loader_val))[0].cuda()

In [56]:
torch.onnx.export(model,	     # model being run​
                    x, 	             # model input ​
                    "model.onnx",                # where to save the model
                    opset_version=12,             # the ONNX version to export the model to​
                    do_constant_folding=True,     # execute constant folding for optimization​
                    input_names = ['input'],      # the models input names​
                    output_names = ['output'],    # the models output names​
                    dynamic_axes={'input' : {0 : 'batch_size'}, 'output' : {0 : 'batch_size'}})

In [57]:
import onnx
from onnx import shape_inference
model = onnx.load("model.onnx")
model_with_shapes = onnx.shape_inference.infer_shapes(model)
onnx.save(model_with_shapes, "model_with_shapes.onnx")